In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/iamthuya/google-cloud-workshops/blob/main/ai-agents/gemini-2-0/building_an_agentic_stock_analysis_system.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fiamthuya%2Fgoogle-cloud-workshops%2Fmain%2Fai-agents%2Fgemini-2-0%2Fbuilding_an_agentic_stock_analysis_system.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/gemini-2-0/building_an_agentic_stock_analysis_system.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/gemini-2-0/building_an_agentic_stock_analysis_system.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  
</table>

# Prepare Environment

## Install required packages

In [ ]:
%pip install --upgrade --quiet --user google-genai

## Import necessary libraries

Run the cell below to import all Python libraries that will be used in this lab.

In [ ]:
import os
import requests

from google import genai
from google.genai.types import (
    GenerateContentConfig,
    GoogleSearch,
    Tool,
)

from IPython.display import Markdown, display

## Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

## Initialize Google GenAI Client

Use the new Google GenAI SDK to establish a client connection to the Gemini API.

In [ ]:
if "google.colab" in sys.modules:
    PROJECT_ID = ""  # @param {type:"string", placeholder: "replace-with-your-project-id", isTemplate: true}
    assert PROJECT_ID != "replace-with-your-project-id", "Please input your project id"
else:
    PROJECT_ID = str(os.environ['GOOGLE_CLOUD_PROJECT'])

LOCATION = os.environ.get('GOOGLE_CLOUD_REGION', 'us-central1')

client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION,
)

## Test Google GenAI Client

Test the client's connection to the Gemini API by asking a question.

In [ ]:
model = "gemini-2.0-flash-001"

response = client.models.generate_content(
    model=model,
    contents="What's the thrid planet from the sun in our solar system?"
)

display(Markdown(response.text))

# Function Calling Recap

To refresh your understanding of function calling, explore Gemini 2.0's enhanced Automatic Function Calling feature through a practical exercise in this section.

Ask Gemini about real-time data like currency exchange rate.

In [ ]:
response = client.models.generate_content(
    model=model,
    contents="What's exchange rate from USD to SGD today?",
)

display(Markdown(response.text))

As you observed above, instead of answering an outdated value, Gemini recommended to check online sources to get an answer.

To make Gemini answers real-time currency exchange rates, we can use Gemini's function calling feature to execute a Python function that queries an external API.

In [ ]:
# A function to get exchange rate
def get_fx_rate(base_currency: str, target_currency: str):
    """
    Fetches the current exchange rate between two currencies.

    Args:
        base_currency: The base currency (e.g., 'USD').
        target_currency: The target currency (e.g., 'SGD').

    Returns:
        The exchange rate information as a json response,
        or None if the rate could not be fetched.
    """

    url = f"https://hexarate.paikama.co/api/rates/latest/{base_currency}?target={target_currency}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()

# Test the function
get_fx_rate('USD', 'SGD')

Here is how to make this function accessible to Gemini, allowing it to function as a tool for currency exchange rate extraction.

In [ ]:
response = client.models.generate_content(
    model=model,
    contents="What's exchange rate from USD to SGD today?",
    config=GenerateContentConfig(
        tools=[get_fx_rate],
    ),
)

display(Markdown(response.text))

Now Gemini can answer the exchange rate using the provided tool aka the `get_fx_rate()` function. That's the power of function calling feature.

# Agentic Stock Analysis System

Agentic Stock Analysis System is a system that automates in-depth market research and stock evaluation, delivering actionable insights with minimal user input.

This system consists of a set of specialized agents: Planner, Researcher, Reporter, and Editor. Together those agents will create a comprehensive report about a requested stock. The Researcher agent leverages Google Search capabilities to gather relevant and up-to-date facts, ensuring thorough analysis.

You will be building the agent individually using Gemini API from sketch.


In [ ]:
stock_ticker = "GOOG"

> "If you fail to plan, you are planning to fail" ~ Benjamin Franklin

As the quote said, the first thing an agentic system should do is to plan necessary tasks to complete the request.

In [ ]:
# 1. Planner Agent

planner_model = "gemini-2.0-flash-lite-001"

planner_prompt = f"Stock Ticker: {stock_ticker}"

planner_system_instruction = """
    You are a tasks planning agent for a stock anlaysis system.
    Your role is to take a stock code as input and break it down into a sequence of specific, important topics for a comprehensive stock analysis.
    You must provide the tasks in a numbered list. Consider the need for data retrieval, analysis, and reporting.
    Include tasks for historical price analysis, financial statement analysis, news sentiment analysis, competitor analysis, and risk assessment.
    Do not carry out the tasks yourself. Only list the tasks. Do not add any introductory text.
"""

planner_response = client.models.generate_content(
    model=planner_model,
    contents=planner_prompt,
    config=GenerateContentConfig(
        system_instruction=planner_system_instruction,
    ),
)

display(Markdown(planner_response.text))

Now we have a list of topics to research and task to carry out. Let's create an agent that uses Google Search to gather information about them and provide a simple write up.

In [ ]:
# 2. Research Agent

researcher_model = "gemini-2.0-flash-001"

researcher_prompt = f"Topics: {planner_response.text}"

researcher_system_instruction = """
    You are a research agent for a stock anlaysis system.
    Your purpose is to use Google Search to find relevant information for all the topics provided.
    Find as many information as you can.
"""

google_search_tool = Tool(google_search=GoogleSearch())

researcher_response = client.models.generate_content(
    model=researcher_model,
    contents=researcher_prompt,
    config=GenerateContentConfig(
        system_instruction=researcher_system_instruction,
        tools=[google_search_tool],
    ),
)

display(Markdown(researcher_response.text))


Although we have the necesary information from the internet, it would be great to have a proper and comprehensive report. Let's create another agent that can do that.

In [ ]:
# 3. Writer Agent

writer_model = "gemini-2.0-flash-thinking-exp-01-21"

writer_prompt = f"Research: {researcher_response.text}"

writer_system_instruction = """
    You are a writer agent for a stock anlaysis system.
    Your purpose is to only use the information provided and write a comprehensive report.
    Make sure to expend on all the topics from the provided information.
"""

writer_response = client.models.generate_content(
    model=writer_model,
    contents=writer_prompt,
    config=GenerateContentConfig(
        system_instruction=writer_system_instruction,
    ),
)

display(Markdown(writer_response.text))

The output is already pretty good. However, some sentences are quite a mouthful to read. Let's create another agent that can edit the report in a way that is easier to read. We will also ask the agent to fact check it with google search tool while it edits.

In [ ]:
# 4. Editor Agent

editor_model = "gemini-2.0-flash-001"

editor_prompt = f"Report: {writer_response.text}"

editor_system_instruction = """
    You are an editor agent for a stock anlaysis system.
    Your purpose is to correct the grammer, sentence structure, and making the report easier to read.
    Do not add any addtional information. Make sure to keep all the information from the provided information.
    You can use Google Search to fact check all provided information and correct as necessary.
    Also provide what you corrected (if any) at the end of the revised report.
"""

editor_response = client.models.generate_content(
    model=editor_model,
    contents=editor_prompt,
    config=GenerateContentConfig(
        system_instruction=editor_system_instruction,
        tools=[google_search_tool],
    ),
)

display(Markdown(editor_response.text))

Congratulations! You've successfully completed the lab. Now, feel free to explore further by changing the stock ticker and analyzing other stocks. You can also consider expending this by adding more agents, e.g. Vistualizer Agent that can plot charts and graph.